In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-10-01


Found rundate 2024-09-30 10:00:04 < 2024-10-01 at commit 02e4cf55; returning commit c59951a


'2024-10-01 10:00:04'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-10-01 10:00:04, NJSP was reporting 486 YTD deaths
Current YTD Deaths (2025-10-01 10:00:02-04:00): 419
Previous year YTD Deaths (adjusted; 2024-10-01 10:00:04): 485.55531475659484
Projected 2025 total: 597.0719439635459
-13.7% change


771d9e2dd7e6f4370df949758f0a0d58f3323d7a: FAUQStats cache miss: 2024, Tue Oct 01 10:00:04 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,East Oak St,NaN,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,647,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13461,02,Bergen,0212,East Rutherford Bor,3,3,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-09-27 00:03:00-04:00
13459,01,Atlantic,0120,Port Republic City,444,Garden State Parkway MP 46.6,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2024-09-28 00:26:00-04:00
13457,06,Cumberland,0614,Vineland City,552,County 552,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2024-09-28 00:34:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,16,3,6,0,24
Bergen,17,1,4,0,22
Burlington,17,5,6,0,26
Camden,16,2,11,2,30
Cape May,7,3,0,1,10
Cumberland,9,2,4,0,14
Essex,10,3,21,1,35
Gloucester,15,4,3,0,19
Hudson,7,1,4,1,11


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

74.9% through the year, 25.1% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,23,31,16
Bergen,driver,14,14,17
Burlington,driver,24,31,17
Camden,driver,13,19,16
Cape May,driver,4,8,7
...,...,...,...,...
Salem,crashes,12,14,12
Somerset,crashes,12,18,8
Sussex,crashes,10,12,9


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,35,0,22,3,9
Bergen,30,0,17,1,9
Burlington,36,0,22,5,13
Camden,38,3,23,3,12
Cape May,21,1,13,4,0
Cumberland,18,0,10,2,8
Essex,50,1,15,10,30
Gloucester,34,0,22,10,8
Hudson,18,1,9,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)